#### 1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

Давайте поработаем с набором данных с платформы kaggle https://www.kaggle.com/adammaus/predicting-churn-for-bank-customers по оттоку клиентов банка

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

Ссылка на google drive: https://drive.google.com/file/d/1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91

In [2]:
!wget 'https://drive.google.com/uc?export=download&id=1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91' -O churn_data.csv

--2023-07-21 14:50:39--  https://drive.google.com/uc?export=download&id=1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91
Resolving drive.google.com (drive.google.com)... 142.250.180.206
Connecting to drive.google.com (drive.google.com)|142.250.180.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/chds0c5mect5ua7ugrbg7mdv73asmh0d/1689943800000/14904333240138417226/*/1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91?e=download&uuid=763d64cc-9986-472d-83e6-900f6eb32b3f [following]
--2023-07-21 14:50:40--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/chds0c5mect5ua7ugrbg7mdv73asmh0d/1689943800000/14904333240138417226/*/1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91?e=download&uuid=763d64cc-9986-472d-83e6-900f6eb32b3f
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 142.250.180.225
Connecting to doc-14-c0-docs.googleuserc

In [3]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [4]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Давайте построим модель. Сразу же будем работать с использованием sklearn pipeline

In [5]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Exited']), df['Exited'], random_state=0)

- Категориальные признаки закодируем с помощью OneHotEncoding
- Вещественные оставим пока как есть

In [6]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [7]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Зададим списки признаков

In [8]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

Теперь нам нужно под каждый признак создать трансформер и объединить их в список.

In [10]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [11]:
feats = FeatureUnion(final_transformers)

Теперь у нас есть пайплайн, который готовит признаки для моделирования.

Добавим модель

In [12]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', CatBoostClassifier(random_state=7575, verbose = False)),
])

#### 2. Обучить любой классификатор (какой вам нравится)

In [13]:
# обучим наш пайплайн
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                                  NumberSelector(key='Age'))])),
                                                ('Balance',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Balance'))])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts'))])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary'))]))])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x14aa9da80>)])

In [14]:
# наши прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.45689952, 0.26569411, 0.12164552, 0.0470462 , 0.01866689,
       0.84131719, 0.02065081, 0.13071489, 0.21687159, 0.85953796])

Также нам нужно от вероятностей перейти к меткам классов. Для этого нужно подобрать порог, после которого мы считаем, что объект можно отнести к классу 1 (если вероятность больше порога - размечаем объект как класс 1, если нет - класс 0)

In [15]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [16]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [17]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.3995483372475593, F-Score=0.644, Precision=0.669, Recall=0.621


In [18]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8755046037226458

In [19]:
row_ctb = {
    'model': 'supervised',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}

metrics_df = pd.concat([metrics_df, pd.DataFrame(row_ctb, index=[0])], ignore_index=True)
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,supervised,0.399548,0.644241,0.669492,0.620825,0.875505


#### 3. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные примеры (класс 1), а только лишь часть

In [20]:
mod_data = X_train.copy()
mod_data['Exited'] = y_train
mod_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2967,2968,15595324,Daniels,579,Germany,Female,39,5,117833.30,3,0,0,5831.00,1
700,701,15803457,Hao,750,France,Female,32,5,0.00,2,1,0,95611.47,0
3481,3482,15644686,Kennedy,729,Spain,Female,34,9,53299.96,2,1,1,42855.97,0
1621,1622,15777797,Kovalyova,689,Spain,Male,38,5,75075.14,1,1,1,8651.92,1
800,801,15747542,Perez,605,France,Male,52,7,0.00,2,1,1,173952.50,0


In [21]:
# get the indices of the positives samples
pos_ind = mod_data[mod_data['Exited'] == 1].sample(frac=1, random_state=42).index

# leave just 25% of the positives marked
perc = 0.25
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 382/1528 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [22]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    7118
 1     382
Name: class_test, dtype: int64


In [23]:
mod_data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,class_test
2967,2968,15595324,Daniels,579,Germany,Female,39,5,117833.30,3,0,0,5831.00,1,-1
700,701,15803457,Hao,750,France,Female,32,5,0.00,2,1,0,95611.47,0,-1
3481,3482,15644686,Kennedy,729,Spain,Female,34,9,53299.96,2,1,1,42855.97,0,-1
1621,1622,15777797,Kovalyova,689,Spain,Male,38,5,75075.14,1,1,1,8651.92,1,-1
800,801,15747542,Perez,605,France,Male,52,7,0.00,2,1,1,173952.50,0,-1


#### 4. Применить random negative sampling для построения классификатора в новых условиях

In [24]:
mod_data = mod_data.sample(frac=1, random_state=42)


data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1, random_state=42)

(382, 15) (382, 15)


In [25]:
sample_train

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,class_test
2421,2422,15574842,Lorenzo,653,Germany,Female,25,2,158266.42,3,1,1,199357.24,0,-1
5828,5829,15684419,Wallace,709,Spain,Female,37,8,0.00,3,1,0,71738.56,0,-1
1117,1118,15592583,Colman,731,France,Female,47,1,115414.19,3,0,0,191734.67,1,1
6303,6304,15591169,Hawes,788,Germany,Female,49,4,137455.99,1,1,0,184178.29,1,-1
7645,7646,15698522,Thomas,660,Germany,Male,39,9,134599.33,2,1,0,183095.87,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7042,7043,15617348,Uchechukwu,544,France,Male,44,1,0.00,2,0,0,69244.24,0,-1
9086,9087,15743545,Nworie,647,Spain,Female,29,2,0.00,2,1,0,179032.68,0,-1
6516,6517,15775797,Esposito,607,Spain,Female,32,7,0.00,3,0,1,10674.62,0,-1
7483,7484,15784526,Chen,616,France,Male,44,5,102016.38,1,0,1,178235.37,1,1


In [26]:
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

pipeline = Pipeline([
    ('features', feats),
    ('classifier', CatBoostClassifier(random_state=7575, verbose = False)),
])


pipeline.fit(sample_train.drop(columns=['class_test', 'Exited']), 
             sample_train['class_test'])

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                                  NumberSelector(key='Age'))])),
                                                ('Balance',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Balance'))])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts'))])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary'))]))])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x14aa9f640>)])

In [27]:
# наши прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.66621146, 0.53534832, 0.47488124, 0.44157838, 0.25768812,
       0.77179711, 0.07028507, 0.14303194, 0.26080378, 0.80375133])

In [28]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.6181770556881959, F-Score=0.579, Precision=0.543, Recall=0.619


In [29]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8246204186027695

In [30]:
row_ctb_pu = {
    'model': 'pu-learning',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}

metrics_df = pd.concat([metrics_df, pd.DataFrame(row_ctb_pu, index=[0])], ignore_index=True)

#### 5. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик)

In [31]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,supervised,0.399548,0.644241,0.669492,0.620825,0.875505
1,pu-learning,0.618177,0.578512,0.543103,0.618861,0.824620


#### 6. *Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [32]:
from tqdm import tqdm

metrics_df = pd.DataFrame(columns=['frac', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])


for frac in tqdm(np.linspace(0.1, 1, 9)):
    mod_data = X_train.copy()
    mod_data['Exited'] = y_train
    mod_data.head()

    # get the indices of the positives samples
    pos_ind = mod_data[mod_data['Exited'] == 1].sample(frac=1, random_state=42).index

    pos_sample_len = int(np.ceil(frac * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    
    mod_data = mod_data.sample(frac=1, random_state=42)


    data_N = mod_data[mod_data['class_test'] == -1]
    data_P = mod_data[mod_data['class_test'] == 1]

    neg_sample = data_N[:data_P.shape[0]]
    pos_sample = data_P.copy()

    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1, random_state=42)

    sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

    pipeline = Pipeline([
        ('features', feats),
        ('classifier', CatBoostClassifier(random_state=7575, verbose = False)),
    ])

    pipeline.fit(sample_train.drop(columns=['class_test','Exited']), 
                 sample_train['class_test'])
    
    # наши прогнозы для тестовой выборки
    preds = pipeline.predict_proba(X_test)[:, 1]
    preds[:10]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, preds)

    
    row_new = {
    'frac': frac,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
    }
    metrics_df = pd.concat([metrics_df, pd.DataFrame(row_new, index=[0])], ignore_index=True)

metrics_df


100%|██████████| 9/9 [00:13<00:00,  1.49s/it]


,frac,thresh,F-Score,Precision,Recall,ROC AUC
0,0.1000,0.625574,0.547152,0.521352,0.575639,0.790514
1,0.2125,0.616225,0.581353,0.543590,0.624754,0.824666
2,0.3250,0.587143,0.593939,0.530960,0.673870,0.839405
3,0.4375,0.667392,0.611718,0.618474,0.605108,0.852184
4,0.5500,0.674056,0.609207,0.607422,0.611002,0.855194
5,0.6625,0.642361,0.630515,0.592401,0.673870,0.862454
6,0.7750,0.707326,0.642424,0.661123,0.624754,0.867754
7,0.8875,0.649761,0.650866,0.607143,0.701375,0.872115
8,1.0000,0.766866,0.647619,0.701835,0.601179,0.873508
